In [15]:
import numpy as np
import tensorflow as tf
import scipy.misc
import scipy.io
import random
from tensorflow.examples.tutorials import mnist


The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [27]:
dataset_loaded = 'mnist' #svhn

In [20]:
############################################ Reading the SVHM training data ################################
svhm_dataset_mat = "./../Datasets/SVHN/train_32x32.mat"
svhn = scipy.io.loadmat(svhm_dataset_mat)

In [26]:
svhn_data = svhn['X']
svhn_data.shape

(32, 32, 3, 73257)

In [2]:
############################################ Reading the MNIST training data ################################
data = mnist.input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
############################################ Reading the SVHM training data ################################


In [3]:
(data.train.next_batch(100))[1].shape

(100, 10)

In [4]:
############################################# Useful funtions ########################################
def ims(name, img):
    scipy.misc.toimage(img, cmin=0, cmax=1).save(name)
    
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))
    #print h, w
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx / size[1]
        #print i,j
        img[j*h:(j*h)+h, i*w:(i*w)+w] = image

    return img

In [5]:
############################################# Model Parameters ########################################

#For MNIST
A, B = 28, 28
img_size = A * B # IMage size/Canvas Size
batch_size = 64 #Batch Size
read_att = True #Whether read attention used
write_att = True #Whether write attention used
enc_size = 256 #Hidden units in LSTM for enc
dec_size = 256 #Hidden units in LSTM for dec
z_size = 10 #QSampler Output size
g_read = 5 #Glimpse dim to read width, height
g_write = 5
seq_len = 10 #Sequence length for the backporp

#For SVHN
'''
A, B = 28, 28
img_size = A * B # IMage size/Canvas Size
batch_size = 64 #Batch Size
read_att = True #Whether read attention used
write_att = True #Whether write attention used
enc_size = 256 #Hidden units in LSTM for enc
dec_size = 256 #Hidden units in LSTM for dec
z_size = 10 #QSampler Output size
g_read = 5 #Glimpse dim to read width, height
g_write = 5
seq_len = 10 #Sequence length for the backporp
'''

learning_rate = 1e-3 #Learning rate
epsilon = 1e-8
epochs = 10000
SHARE = None

canvas = [0] * seq_len #Canvases used for writing
mu, sigma, logsigma = [0] * seq_len, [0] * seq_len,[0] * seq_len #Used in Loss function

In [6]:
############################################# Model funtions ##########################################
#Affine transormation y = W(x) + b, where x is batch_size * feature_size
def linearOP(x, op_size):
    W = tf.get_variable("W", shape = [x.get_shape()[1], op_size])
    b = tf.get_variable("b", shape = op_size ,initializer=tf.constant_initializer(0.0))
    return tf.matmul(x,W)+b

#Encoder step 
def encode(input,state):
    with tf.variable_scope("encode", reuse = SHARE):
        return enc(input, state)

#Decoder Step
def decode(input, state):
    with tf.variable_scope("decode", reuse = SHARE):
        return dec(input, state)
    
#Sample z_t ~ Q(Z|h_enc) from reparametarization trick
def QSample(h_enc):
    with tf.variable_scope("Mu", reuse = SHARE):
        mu = linearOP(h_enc, z_size)
    with tf.variable_scope("sigma", reuse = SHARE):
        logsigma = linearOP(h_enc, z_size)
        sigma = tf.exp(logsigma)
    return (mu + sigma * sdn),mu, sigma, logsigma

#Applying binary cross ent. between two distn
def binary_crossentropy(t,o):
    return -(t*tf.log(o+epsilon) + (1.0-t)*tf.log(1.0-o+epsilon))


# given a hidden decoder layer; locate where to put attention filters
def attn_window(scope, h_dec, N):
    with tf.variable_scope(scope, reuse=SHARE):
        parameters = linearOP(h_dec, N)
    # gx_, gy_: center of 2d gaussian on a scale of -1 to 1
    gx_, gy_, log_sigma2, log_delta, log_gamma = tf.split(1,5,parameters)

    # move gx/gy to be a scale of -imgsize to +imgsize
    gx = (A+1)/2 * (gx_ + 1)
    gy = (B+1)/2 * (gy_ + 1)

    sigma2 = tf.exp(log_sigma2)
    # stride/delta: how far apart these patches will be
    delta = (max(A,B) - 1) / ((N-1) * tf.exp(log_delta))
    # returns [Fx, Fy, gamma]
    return filterbank(gx,gy,sigma2,delta,N) + (tf.exp(log_gamma),)
    
    
# Given a center, distance, and spread, Construct [N x N] patches of gaussian filters
# represented by Fx = horizontal gaussian, Fy = vertical guassian
def filterbank(gx, gy, sigma2, delta,N):
    # 1 x N, look like [[0,1,2,3,4]]
    grid_i = tf.reshape(tf.cast(tf.range(N), tf.float32),[1, -1])
    # centers for the individual patches
    mu_x = gx + (grid_i - N/2 - 0.5) * delta
    mu_y = gy + (grid_i - N/2 - 0.5) * delta
    mu_x = tf.reshape(mu_x, [-1,N, 1])
    mu_y = tf.reshape(mu_y, [-1,N, 1])
    # 1 x 1 x imgsize, looks like [[[0,1,2,3,4,...,27]]]
    a = tf.reshape(tf.cast(tf.range(A), tf.float32), [1, 1, -1])
    b = tf.reshape(tf.cast(tf.range(B), tf.float32), [1, 1, -1])
    # list of gaussian curves for x and y
    sigma2 = tf.reshape(sigma2, [-1, 1, 1])
    Fx = tf.exp(-tf.square((a - mu_x) / (2*sigma2)))
    Fy = tf.exp(-tf.square((b - mu_x) / (2*sigma2)))# batch x N x B
    # normalize so area-under-curve = 1
    Fx = Fx / tf.maximum(tf.reduce_sum(Fx,2,keep_dims=True),epsilon)
    Fy = Fy / tf.maximum(tf.reduce_sum(Fy,2,keep_dims=True),epsilon)
    return Fx, Fy

In [7]:
############################################ Model Details#############################################
x = tf.placeholder(tf.float32, [batch_size, img_size])
# Qsampler noise which is Std. Normal, used in geting z_t via reparametization of this shape
sdn = tf.random_normal((batch_size,z_size), mean=0, stddev=1) 
enc = tf.nn.rnn_cell.LSTMCell(num_units=enc_size, state_is_tuple=True)
dec = tf.nn.rnn_cell.LSTMCell(num_units=dec_size, state_is_tuple=True)

In [8]:
#Read operation with/without attention
def read(x, xhat, h_dec_prev):
    if(read_att == False):
        return tf.concat(1, [x, xhat])
    elif(read_att == True):
        Fx, Fy, gamma = attn_window("read", h_dec_prev, g_read)    
        #Apply the filter over the img(batch_size * img_size)
        def filter_img(img, Fx, Fy, gamma, N):
            Fxt=tf.transpose(Fx,perm=[0,2,1])
            img=tf.reshape(img,[-1,B,A])
            # Apply the gaussian patches:
            # A = B = imgsize (they are all the image size)
            # attn = height/length of attention patches
            # allfilters = [attn, vert] * [imgsize,imgsize] * [horiz, attn] (Eq 27)
            # we have batches, so the full batch_matmul equation looks like:
            # [1, 1, vert] * [batchsize,imgsize,imgsize] * [1, horiz, 1]
            glimpse=tf.batch_matmul(Fy,tf.batch_matmul(img,Fxt))
            glimpse=tf.reshape(glimpse,[-1,N*N])
            return glimpse*tf.reshape(gamma,[-1,1])
        x = filter_img(x, Fx, Fy, gamma, g_read) #batch_size * (g_read * g_read)
        xhat = filter_img(xhat, Fx, Fy, gamma, g_read)
        return tf.concat(1, [x, xhat]) #Concatinating all the features together


In [9]:
#Write operation with/without attention
def write(h_dec):
    if(write_att == False):
        with tf.variable_scope("writeWithoutAtten", reuse = SHARE):
            return linearOP(x, img_size)
    elif(write_att == True):
        with tf.variable_scope("writeAtten", reuse=SHARE):
            w = linearOP(h_dec, g_write*g_write)
        N=g_write
        w = tf.reshape(w, [batch_size, N,N])
        Fx, Fy, gamma = attn_window("write", h_dec, g_write)
        Fyt = tf.transpose(Fy, perm=[0,2,1])
        # [vert, attn_n] * [attn_n, attn_n] * [attn_n, horiz]
        wr = tf.batch_matmul(Fyt, tf.batch_matmul(w, Fx))
        wr = tf.reshape(wr, [batch_size, B*A])
        return wr * tf.reshape(1.0/gamma, [-1, 1])

In [10]:
#Initial State Variables
h_dec_prev = tf.zeros([batch_size, dec_size])
dec_state = dec.zero_state(batch_size, tf.float32)
enc_state = enc.zero_state(batch_size, tf.float32)


In [11]:
#Progress through time steps, equations defined in paper
for t in range(seq_len):
    c_prev = tf.zeros([batch_size, img_size]) if t == 0 else canvas[t-1]
    xhat = x - tf.sigmoid(c_prev) #error img = org iamge - generated img
    r_t = read(x, xhat, h_dec_prev) #read the cropped/full image
    h_enc, enc_state = encode(tf.concat(1, [r_t, h_dec_prev]),enc_state)
    z_t, mu[t], sigma[t], logsigma[t] = QSample(h_enc)
    h_dec, dec_state = decode(z_t,dec_state)
    canvas[t] = c_prev + write(h_dec)
    h_dec_prev = h_dec
    SHARE = True #USe previously made variables

In [12]:
#Loss funtion implementation
xdash = tf.sigmoid(canvas[-1]) #Squissh the loss from -1 to 1, then take bin cross_entropy for batch
Lx = tf.reduce_mean(tf.reduce_sum(binary_crossentropy(x, xdash), 1))

kl_ = [0] *  seq_len
for t in range(seq_len):
    mu_t = tf.square(mu[t])
    sigma_t = tf.square(sigma[t])
    logsigma_t = logsigma[t]
    kl_[t] = 0.5*tf.reduce_sum(mu_t+sigma_t-2*logsigma_t,1)#Corressponding to 1 x batch_size
KL = tf.add_n(kl_)#Summing up kl terms from 1:T
KL = KL - seq_len * 0.5
Ly = tf.reduce_mean(KL)

totalLoss = Lx + Ly

In [13]:
#Optimizer
optimizer=tf.train.AdamOptimizer(learning_rate, beta1=0.5)
grads=optimizer.compute_gradients(totalLoss)
for i,(g,v) in enumerate(grads):
    if g is not None:
        grads[i]=(tf.clip_by_norm(g,5),v) # clip gradients
train_op=optimizer.apply_gradients(grads)

In [14]:
#Training model
sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in xrange(epochs):
    X_train, _ = data.train.next_batch(batch_size)
    feed_dict = {x: X_train}
    cs, gen_loss, lat_loss, _ = sess.run([canvas, Lx, Ly, train_op], feed_dict=feed_dict)
   
    if i % 500 == 0:
        print "iter %d genloss %f latloss %f" % (i, gen_loss, lat_loss)
        cs = 1.0/(1.0+np.exp(-np.array(cs))) # x_recons=sigmoid(canvas)
        for cs_iter in xrange(seq_len):
            results = cs[cs_iter]
            results_square = np.reshape(results, [-1, A, B])
            #print results_square.shape
            ims("results/"+str(i)+"-step-"+str(cs_iter)+".jpg",merge(results_square,[8,8]))

sess.close()

iter 0 genloss 544.305115 latloss 46.437206
iter 500 genloss 193.861420 latloss 46.887955
iter 1000 genloss 171.391205 latloss 47.238720


/home/peyush/tEnv/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: overflow encountered in exp


iter 1500 genloss 155.149567 latloss 47.900330
iter 2000 genloss 149.505829 latloss 47.444664
iter 2500 genloss 143.341888 latloss 47.010830
iter 3000 genloss 129.464523 latloss 47.433628
iter 3500 genloss 125.463028 latloss 47.291893
iter 4000 genloss 121.374557 latloss 47.369228
iter 4500 genloss 107.398605 latloss 47.288170
iter 5000 genloss 106.758438 latloss 47.087303
iter 5500 genloss 101.534348 latloss 47.025978
iter 6000 genloss 89.629601 latloss 46.828667
iter 6500 genloss 91.066574 latloss 46.933029
iter 7000 genloss 88.702499 latloss 46.593971
iter 7500 genloss 83.561974 latloss 46.528999
iter 8000 genloss 78.266006 latloss 46.581345
iter 8500 genloss 87.610870 latloss 46.578934
iter 9000 genloss 81.244064 latloss 46.488026
iter 9500 genloss 82.113327 latloss 46.499725
